<a href="https://colab.research.google.com/github/google-research/skai/blob/skai-colab-0000006/src/colab/Load_SKAI_Colab_MaxarImagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load SKAI Colab Maxar Imagery
This notebook aims to facilitate the loading, visuqlization, sorting, selection, filtering and downloading of the satelite imagery open sourced by [Maxar's Open Data Program](https://www.maxar.com/open-data), open data for select sudden onset major crisis events.

We leverage on the open source python library [Leafmap](https://leafmap.org/notebooks/67_maxar_open_data/) that offers functions to retrieve and dowload pre- and post-disaster imagery collection provided by Maxar.

To use:
1. Open this notebook in Colab.
2. Connect to your python kernel (hosted or local runtime).
3. Change settings as appropriate.
4. Run code cells.

In [ ]:
# @title Settings - Run this cell first

# Install required packages
!pip install --q fastapi kaleido python-multipart uvicorn tensorflow-probability==0.23.0
!pip install --q cohere openai tiktoken
!pip install --q leafmap geopandas cogeo-mosaic
!pip install --q ipywidgets

# Enable ipywidgets extension
!jupyter nbextension enable --py widgetsnbextension

# Import libraries
import os
from datetime import timezone
import pandas as pd
import datetime
import numpy as np

import leafmap
import leafmap.foliumap as leafmap_fol
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import Polygon

import ipywidgets as widgets
from IPython.display import display

import json
import shutil
from typing import Sequence
import urllib.parse

from absl import app, flags
from bs4 import BeautifulSoup
import requests
import tensorflow as tf
from tqdm import tqdm

from google.cloud import storage
from google.colab import auth

# Get the list of collections available on MX open data program
collection_list = leafmap.maxar_collections()

# Definition of the dropdown widget to select the collection of satellite imagery
collection_picker = widgets.Dropdown(
    options=collection_list,
    value=collection_list[0]
)
out_col_pick = widgets.Output()

# Definition of the date picker widget to set the date of the disaster
disaster_date = widgets.DatePicker(
    description='Select disaster date',
    style={'description_width': 'initial'}
)

# Definition of the text box widget to set the path to the AOI file
text_aoi = widgets.Text(
    description="Enter path to geojson aoi file (colab path or gsutil URI)",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='800px')
)

# Definition of the button widget to activate the loading of the AOI file
btn_aoi = widgets.Button(description='Upload AOI')
out_btn_aoi = widgets.Output()


class MapExplorer:

    def __init__(self, mxr_coll):
        """
        Initializes the MapExplorer class.

        Parameters:
        - mxr_coll (str): The initial Maxar collection.
        """
        self.mxr_coll = mxr_coll
        self.child_collections = leafmap.maxar_child_collections(mxr_coll)
        self.geodataframe = gpd.read_file(leafmap.maxar_collection_url(mxr_coll, dtype='geojson'))
        self.disaster_date = None
        self.area_of_interest = None
        self.gdf_aoi=None

    def setup_geodataframe(self):
        """
        Sets up the GeoDataFrame by fetching child collections and loading GeoJSON data.
        """
        out_col_pick.clear_output()
        with out_col_pick:
            display(f"Loading of all the available child collection IDs and metadata from event {self.mxr_coll} ...")
        child_collections = leafmap.maxar_child_collections(self.mxr_coll)
        geodataframe = gpd.read_file(leafmap.maxar_collection_url(self.mxr_coll, dtype='geojson'))
        geodataframe['geometry_maxar'] = geodataframe.geometry
        self.child_collections = child_collections
        self.geodataframe = geodataframe
        out_col_pick.clear_output()
        with out_col_pick:
          display(f"The number of child collections for event {self.mxr_coll}: {len(set(self.child_collections))}",
                  f"First date: {min(self.geodataframe.datetime)}, Last date: {max(self.geodataframe.datetime)}")

    def setup_date_and_aoi(self):
        """
        Sets up date and area of interest, creating a map instance with appropriate layers.

        Returns:
        - leafmap_fol.Map: The configured map instance.
        """
        if self.disaster_date is not None:
            self.geodataframe['disaster_phase'] = ['pre_disaster' if x < self.disaster_date else 'post_disaster' for x in self.geodataframe.datetime]
            post_footprint = gpd.GeoDataFrame({'id': [1],
                                               'geometry': [self.geodataframe.loc[self.geodataframe.disaster_phase == 'post_disaster'].unary_union]})
            pre_footprint = gpd.GeoDataFrame({'id': [1],
                                              'geometry': [self.geodataframe.loc[self.geodataframe.disaster_phase == 'pre_disaster'].unary_union]})
            map_instance = leafmap_fol.Map()
            map_instance.add_gdf(pre_footprint, layer_name="All Imagery Pre-disaster Footprint",
                                  style={'color': 'green', 'weight': 2, 'fillOpacity': 0.1})
            map_instance.add_gdf(post_footprint, layer_name="All Imagery Post-disaster Footprint",
                                  style={'color': 'red', 'weight': 2, 'fillOpacity': 0.1})
        else:
            all_footprint = gpd.GeoDataFrame({'id': [1], 'geometry': [self.geodataframe.unary_union]})
            map_instance = leafmap_fol.Map()
            map_instance.add_gdf(all_footprint, layer_name="All Imagery Disaster Footprint",
                                  style={'color': 'black', 'weight': 2, 'fillOpacity': 0.1})

        if (self.area_of_interest is not None) and (self.area_of_interest != ''):
            if self.area_of_interest[:5] == 'gs://':
                os.system(f"""gsutil cp {self.area_of_interest} /content/aoi.geojson""")
                self.gdf_aoi = gpd.read_file('/content/aoi.geojson')
            else:
                self.gdf_aoi = gpd.read_file(self.area_of_interest)
            map_instance.add_gdf(self.gdf_aoi, layer_name="Area Of Interest",
                                  style={'color': 'orange', 'weight': 2, 'fillOpacity': 0.05})

        return map_instance

    def initialize_dropdown(self):
        """
        Initializes the dropdown, setting up the GeoDataFrame and creating an initial map instance.
        """
        out_btn_aoi.clear_output()
        self.setup_geodataframe()
        map_instance = self.setup_date_and_aoi()
        with out_btn_aoi:
          display(map_instance)

    def on_dropdown_selection(self, change):
        """
        Handles the selection change in the dropdown, updating the Maxar collection and refreshing the map.
        """
        self.mxr_coll = change.new
        self.disaster_date = None
        self.setup_geodataframe()
        map_instance = self.setup_date_and_aoi()
        out_btn_aoi.clear_output()
        with out_btn_aoi:
          display(map_instance)
        disaster_date.value = self.disaster_date


    def on_date_selection(self, change):
        """
        Handles the date selection change, updating the disaster date and refreshing the map.
        """
        if (change.new is None) or (pd.isnull(pd.to_datetime(change.new, errors='coerce'))):
          self.disaster_date = None
        else:
          self.disaster_date = pd.to_datetime(change.new, errors='coerce').replace(tzinfo=timezone.utc)
        self.setup_geodataframe()
        map_instance = self.setup_date_and_aoi()
        out_btn_aoi.clear_output()
        with out_btn_aoi:
          display(map_instance)

    def on_aoi_selection(self, change):
        """
        Handles the area of interest selection change.
        """
        self.area_of_interest = change.new

    def on_download_button_click(self, change):
        """
        Handles the download button click, refreshing the map.
        """
        out_btn_aoi.clear_output()
        self.setup_geodataframe()
        map_instance = self.setup_date_and_aoi()
        with out_btn_aoi:
            display(map_instance)

def preprocess_aoi_data(map_explorer):
    """
    Preprocesses Area of Interest (AOI) data based on the selected AOI file or region.

    Args:
        map_explorer (MapExplorer): An instance of the MapExplorer class.

    Returns:
        GeoDataFrame: Processed AOI data.
    """
    if not (map_explorer.area_of_interest == '' or map_explorer.area_of_interest is None):
        intersecting_aoi_all = map_explorer.geodataframe.sjoin(map_explorer.gdf_aoi, how='inner')
    else:
        intersecting_aoi_all = map_explorer.geodataframe
    return intersecting_aoi_all

def compute_intersection(intersecting_aoi_all):
    """
    Computes the intersection of pre and post-disaster imagery in the AOI.

    Args:
        intersecting_aoi_all (GeoDataFrame): AOI data with pre and post-disaster imagery.

    Returns:
        GeoDataFrame: Data with the intersection of pre and post-disaster imagery.
    """
    intersection_final = intersecting_aoi_all.loc[
        (intersecting_aoi_all.disaster_phase == 'pre_disaster')].sjoin(
        intersecting_aoi_all.loc[(intersecting_aoi_all.disaster_phase == 'post_disaster')],
        how='inner', lsuffix='pre', rsuffix='post')
    return intersection_final

def calculate_overlay_percentage(intersection_final):
    """
    Calculates overlay percentage for pre and post-disaster imagery.

    Args:
        intersection_final (GeoDataFrame): Data with the intersection of pre and post-disaster imagery.

    Returns:
        GeoDataFrame: Data with added overlay percentage columns.
    """
    intersect = intersection_final['geometry_maxar_post'].intersection(
        intersection_final['geometry_maxar_pre']).area
    intersection_final['overlay_pre_post'] = intersect / intersection_final['geometry_maxar_post'].area

    intersection_final['geometry_prec_post'] = intersection_final['geometry_maxar_post'].intersection(
        intersection_final.groupby(['catalog_id_pre', 'catalog_id_post', 'quadkey_post'])[
            'geometry_maxar_pre'].transform(lambda x: unary_union(x)))

    intersect = intersection_final['geometry_maxar_post'].intersection(
        intersection_final['geometry_prec_post']).area
    intersection_final['overlay_prec_post'] = intersect / intersection_final['geometry_maxar_post'].area
    intersection_final['max_overlay_prec_post'] = intersection_final.groupby(['catalog_id_post', 'quadkey_post'])[
        'overlay_prec_post'].transform('max')

    return intersection_final

def calculate_cloud_percentage(intersection_final):
    """
    Calculates cloud percentage for pre and post-disaster imagery.

    Args:
        intersection_final (GeoDataFrame): Data with added overlay percentage columns.

    Returns:
        GeoDataFrame: Data with added cloud percentage columns.
    """
    intersection_final['tile:clouds_percent_prec'] = intersection_final['tile:clouds_percent_pre'].values * \
                                                     intersection_final['overlay_pre_post'].values / \
                                                     intersection_final['overlay_prec_post'].values
    intersection_final['tile:clouds_percent_prec'] = np.minimum(100, intersection_final.groupby(
        ['catalog_id_pre', 'catalog_id_post', 'quadkey_post'])['tile:clouds_percent_prec'].transform('sum'))

    return intersection_final

def logistic_function(x):
    """
    Logistic function used in scoring calculations.

    Parameters:
    - x (float): Input value.

    Returns:
    - float: Transformed value.
    """
    return 2 * (1 / (1 + np.exp(-0.1 * x))) - 1

def calculate_final_score(row, widget):
    """
    Calculate final scores based on input parameters.

    Parameters:
    - row (pd.Series): DataFrame row containing relevant information.
    - widget (MapExplorer): Instance of the MapExplorer class.

    Returns:
    - pd.Series: Updated DataFrame row with calculated scores.
    """
    row['score_tile'] = row['overlay_pre_post'] * ((100 - row['tile:clouds_percent_pre']) / 100) * (1 - logistic_function((widget.disaster_date - row['datetime_pre']).dt.days / 365.25))
    row['score_col'] = row['overlay_prec_post'] * ((100 - row['tile:clouds_percent_prec']) / 100) * (1 - logistic_function((widget.disaster_date - row['datetime_pre']).dt.days / 365.25))
    return row

def recursive_function(group):
    """
    Recursive function for processing groups of data.

    Parameters:
    - group (pd.DataFrame): DataFrame group containing relevant information.

    Returns:
    - list: Resulting information after processing the group.
    """
    result = []
    intermediate = []
    area = []
    idx = 0

    for _, row in group.iterrows():
        if idx == 0:
            idx += 1
            list_ = row['catalog_id_pre']
            result.append(row['geometry_maxar_post'].difference(row['geometry_prec_post']))
            intermediate.append(1)
            area.append(result[-1].area / row['geometry_maxar_post'].area)
        else:
            if row['catalog_id_pre'] == list_:
                result.append(result[-1])
                intermediate.append(intermediate[-1])
                area.append(area[-1])
            else:
                list_ = [row['catalog_id_pre'], row['quadkey_pre']]
                if area[-1] <= 1 - row['area_to_cover']:
                    result.append(result[-1])
                    intermediate.append(0)
                    area.append(area[-1])
                else:
                    diff = result[-1].difference(row['geometry_prec_post'])
                    if diff.is_empty:
                        result.append(result[-1])
                        intermediate.append(0)
                        area.append(area[-1])
                    else:
                        result.append(diff)
                        intermediate.append(1)
                        area.append(result[-1].area / row['geometry_maxar_post'].area)

    return [group.catalog_id_pre, group.quadkey_pre, result, intermediate, area]

def select_tiles(intersection_data):
    """
    Selects and analyzes tiles based on the maximum coverage and best score.

    Args:
        intersection_data (GeoDataFrame): Data with the intersection of pre and post-disaster imagery.

    Returns:
        GeoDataFrame: Selected and analyzed tiles based on criteria.
    """
    # Calculate area_to_cover for each group and merge it with the original dataframe
    selected_tiles = pd.merge(intersection_data, intersection_data.groupby(['catalog_id_post', 'quadkey_post'])[
        'geometry_prec_post', 'geometry_maxar_post'].apply(
        lambda x: unary_union(x['geometry_prec_post']).area / x['geometry_maxar_post'].iloc[0].area).reset_index(
        name='area_to_cover'), on=['catalog_id_post', 'quadkey_post'])

    # Sort the dataframe based on specified criteria
    selected_tiles.sort_values(by=['catalog_id_post', 'quadkey_post', 'score_col', 'catalog_id_pre'],
                               ascending=[True, True, False, True], inplace=True)

    # Apply the recursive_function
    results = selected_tiles.groupby(['catalog_id_post', 'quadkey_post']).apply(recursive_function)
    selected_tiles_expanded = pd.DataFrame(results.reset_index()[0].to_list(),
                                           index=results.index,
                                           columns=['catalog_id_pre', 'quadkey_pre', 'poly_diff', 'select_bool', 'area_perc_diff']).explode(
        ['catalog_id_pre', 'quadkey_pre', 'poly_diff', 'select_bool', 'area_perc_diff']).reset_index()

    # Filter out rows where select_bool is not 1
    selected_tiles_expanded = selected_tiles_expanded.loc[selected_tiles_expanded.select_bool == 1]

    # Merge with the original dataframe
    selected_and_analyzed_tiles = pd.merge(selected_tiles, selected_tiles_expanded[['catalog_id_post', 'quadkey_post', 'catalog_id_pre', 'quadkey_pre']],
                                           on=['catalog_id_post', 'quadkey_post', 'catalog_id_pre', 'quadkey_pre'],
                                           how='inner')

    selected_and_analyzed_tiles['date_pre']=pd.to_datetime(selected_and_analyzed_tiles['datetime_pre']).dt.strftime('%Y%m%d')
    selected_and_analyzed_tiles['date_post']=pd.to_datetime(selected_and_analyzed_tiles['datetime_post']).dt.strftime('%Y%m%d')

    return selected_and_analyzed_tiles

def analyze_tiles(map_explorer, intersecting_aoi_all, selected_tiles):
  """
  Analyzes the number of tiles filtered out during the selection process.

  Args:
      map_explorer (MapExplorer): An instance of the MapExplorer class.
      intersecting_aoi_all (GeoDataFrame): AOI data with pre and post-disaster imagery.
      selected_tiles (GeoDataFrame): Selected tiles based on criteria.

  Returns:
      pre_footprint (GeoDataFrame): Pre-disaster imagery footprints.
      post_footprint (GeoDataFrame): Post-disaster imagery footprints.
      final_pre (GeoDataFrame): Selected pre-disaster imagery tiles.
      final_post (GeoDataFrame): Selected post-disaster imagery tiles.
  """
  initial_pre = intersecting_aoi_all.loc[intersecting_aoi_all.disaster_phase == 'pre_disaster',
                                        ['catalog_id', 'quadkey', 'geometry']].drop_duplicates(keep='first')
  print(f"Total of {len(initial_pre)} pre-disaster imagery tiles initially intersecting AOI")

  initial_post = intersecting_aoi_all.loc[intersecting_aoi_all.disaster_phase == 'post_disaster',
                                          ['catalog_id', 'quadkey', 'geometry']].drop_duplicates(keep='first')
  print(f"Total of {len(initial_post)} post-disaster imagery tiles initially intersecting AOI")

  final_pre = selected_tiles[['catalog_id_pre', 'quadkey_pre', 'geometry_maxar_pre', 'date_pre', 'visual_pre']].drop_duplicates(
      keep='first').set_geometry("geometry_maxar_pre")
  print(
      f"Total of {len(final_pre)} pre-disaster imagery tiles finally selected intersecting AOI and post-disaster imagery tiles")

  final_post = selected_tiles[['catalog_id_post', 'quadkey_post', 'geometry_maxar_post', 'date_post', 'visual_post']].drop_duplicates(
      keep='first').set_geometry("geometry_maxar_post")
  print(
      f"Total of {len(final_post)} post-disaster imagery tiles finally selected intersecting AOI and pre-disaster imagery tiles")

  pre_footprint=gpd.GeoDataFrame({'id':[1],'geometry':[final_pre.unary_union]})
  post_footprint=gpd.GeoDataFrame({'id':[1],'geometry':[final_post.unary_union]})

  return final_pre, final_post, pre_footprint, post_footprint

def visualize_tiles(map_explorer, selected_tiles, pre_footprint, post_footprint, final_pre, final_post):
  """
  Visualizes imagery on a map based on provided GeoDataFrames and widget information.

  Args:
      map_explorer (MapExplorer): An instance of the MapExplorer class.
      pre_footprint (GeoDataFrame): Pre-disaster imagery footprints.
      post_footprint (GeoDataFrame): Post-disaster imagery footprints.
      final_pre (GeoDataFrame): Selected pre-disaster imagery tiles.
      final_post (GeoDataFrame): Selected post-disaster imagery tiles.

  Returns:
      folium.Map: Folium map with the visualized imagery.
      intersection_pre_post_aoi (GeoDataFrame): Selected post-disaster imagery tiles with the intersection of aoi.
  """

  # Check if AOI is provided
  if not (map_explorer.area_of_interest == '' or map_explorer.area_of_interest is None):
      intersection_pre_post_aoi = gpd.GeoDataFrame(post_footprint.intersection(pre_footprint).intersection(map_explorer.gdf_aoi)).set_geometry(0,crs="EPSG:4326")
  else:
      intersection_pre_post_aoi = gpd.GeoDataFrame(post_footprint.intersection(pre_footprint)).set_geometry(0,crs="EPSG:4326")

  # Create a folium map
  m = leafmap_fol.Map()

  # Check if AOI is provided, and add it to the map if available
  if not (map_explorer.area_of_interest == '' or map_explorer.area_of_interest is None):
      m.add_gdf(map_explorer.gdf_aoi, layer_name="Area Of Interest", style={'color': 'orange', 'weight': 2, "fill": False})

  # Add pre-disaster imagery to the map
  m.add_gdf(pre_footprint, layer_name="AOI Imagery Pre-disaster", style={'color': 'green', 'weight': 2, "fill": False})

  # Add post-disaster imagery to the map
  m.add_gdf(post_footprint, layer_name="AOI Imagery Post-disaster", style={'color': 'red', 'weight': 2, "fill": False})

  # Add AOI Imagery Overlap to the map
  m.add_gdf(intersection_pre_post_aoi, layer_name="AOI Imagery Overlap", style={'color': 'blue', 'weight': 2, "fill": False})

  # Add individual pre-disaster tiles to the map with unique styles
  for id in pd.unique(final_pre.catalog_id_pre):
      style = {
          'fillColor': 'green',  # Fill color of polygons
          'color': 'green',  # Border color of polygons
          'weight': 0.1,  # Border thickness
          'fillOpacity': 0.1  # Opacity of the fill color
      }
      m.add_gdf(final_pre.loc[final_pre.catalog_id_pre == id], layer_name=f"{id}_pre", style=style, info_mode='on_click')

  # Add individual post-disaster tiles to the map with unique styles
  for id in pd.unique(final_post.catalog_id_post):
      style = {
          'fillColor': 'red',  # Fill color of polygons
          'color': 'red',  # Border color of polygons
          'weight': 0.1,  # Border thickness
          'fillOpacity': 0.1,  # Opacity of the fill color
      }
      m.add_gdf(final_post.loc[final_post.catalog_id_post == id], layer_name=f"{id}_post", style=style, info_mode='on_click')

  return m, intersection_pre_post_aoi


def check_file_exists(bucket_name, file_name):
    #storage_client = storage.Client.from_service_account_json(pathsys_credentials)
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name.strip("/"))
    return blob.exists()

def download_image(url: str, output_path: str) -> None:
  with requests.get(url, stream=True) as r:
    with tf.io.gfile.GFile(output_path, 'wb') as f:
      shutil.copyfileobj(r.raw, f)

def make_download_path(url: str, output_dir: str) -> str:
  parsed_url = urllib.parse.urlparse(url)
  components = parsed_url.path.split('/')
  image_name = components[-1].split('.')[0]
  return f'{output_dir}/{image_name}-{components[-3]}.tif'

def download_images(urls: Sequence[str], output_dir: str) -> None:
  if output_dir[:5]!='gs://':
    for url in tqdm(urls, total=len(urls)):
      print(make_download_path(url, output_dir))
      download_image(url, make_download_path(url, output_dir))
  else:
    bucket=output_dir.split('/')[2]
    output_dir='/'+'/'.join(output_dir.split('/')[3:])
    counter_k=0
    for url in tqdm(urls, total=len(urls)):
      if check_file_exists(bucket, make_download_path(url, output_dir)):
        print(make_download_path(url, f'gs://{bucket}{output_dir}'),f'file already existing')
      else:
        counter_k+=1
        download_image(url, make_download_path(url, '/content/sample_data'))
        os.system(f"""gsutil cp {make_download_path(url, '/content/sample_data')} {make_download_path(url, f'gs://{bucket}{output_dir}')}""")
        print(make_download_path(url, f'gs://{bucket}{output_dir}'))
      if counter_k>20:
        counter_k=0
        os.system(f"""rm /content/sample_data/*""")
    os.system(f"""rm /content/sample_data/*""")



In [ ]:
#@title Disaster Information

# Create an instance of the MapExplorer class
map_explorer = MapExplorer(collection_picker.value)

# Initialize the dropdown and display the initial map
map_explorer.initialize_dropdown()

# Observe changes in the collection picker, disaster date, text AOI, and button AOI
collection_picker.observe(map_explorer.on_dropdown_selection, names="value")
disaster_date.observe(map_explorer.on_date_selection, names="value")
text_aoi.observe(map_explorer.on_aoi_selection, names="value")
btn_aoi.on_click(map_explorer.on_download_button_click)

# Display the collection picker widget
display(collection_picker)

# Display the output for the collection picker
display(out_col_pick)

# Display the disaster date widget
display(disaster_date)

# Display the text AOI and AOI button widgets
display(text_aoi, btn_aoi)

# Display the output for the AOI button
display(out_btn_aoi)

In [ ]:
# @title Optional - Launch the download of all imagery from Maxar bucket

##@markdown ---
PATH_FILE_FOLDER = "" #@param {type:"string"}
#The destination path of the metadata csv file (colab path or gsutil URI):
PATH_FILE_LIST = os.path.join(PATH_FILE_FOLDER, 'metadata_maxarimagery.csv')

if PATH_FILE_LIST!="":
  if PATH_FILE_LIST[:5]!='gs://':
    with open(PATH_FILE_LIST, 'w') as file:
        map_explorer.geodataframe.to_csv(file)
  else:
    with open('/content/sample_data/imagery_list_maxar.csv', 'w') as file:
        map_explorer.geodataframe.to_csv(file)
    auth.authenticate_user()
    os.system(f"""gsutil cp /content/sample_data/imagery_list_maxar.csv {PATH_FILE_LIST}""")


for catalog_id in pd.unique(map_explorer.geodataframe['catalog_id']):
  df_catalog_id=map_explorer.geodataframe.loc[map_explorer.geodataframe['catalog_id']==catalog_id]
  date_catalog_id=pd.to_datetime(df_catalog_id['datetime']).dt.strftime('%Y%m%d').iloc[0]
  print(f'Download of {df_catalog_id.shape[0]} disaster imagery, catalog_id {catalog_id}')
  PATH_FILE_ID = os.path.join(PATH_FILE_FOLDER, f'{date_catalog_id}_{catalog_id}')
  download_images(df_catalog_id.visual, PATH_FILE_ID)


In [ ]:
# @title Automatic selection and visualization of the intersected pre- and post-disaster footprint

# Preprocess AOI data
intersecting_aoi_all = preprocess_aoi_data(map_explorer)

# Compute intersection of pre and post imagery
intersection_final = compute_intersection(intersecting_aoi_all)

# Calculate overlay percentage
intersection_final = calculate_overlay_percentage(intersection_final)

# Calculate cloud percentage
intersection_final = calculate_cloud_percentage(intersection_final)

# Calculate selection score
intersection_final = calculate_final_score(intersection_final, map_explorer)

# Select tiles based on the maximum coverage and best score
selected_tiles = select_tiles(intersection_final)

# Analyze the number of tiles filtered out
final_pre, final_post, pre_footprint, post_footprint = analyze_tiles(map_explorer, intersecting_aoi_all, selected_tiles)

# Visualize selected tiles footprints
m, intersection_pre_post_aoi  = visualize_tiles(map_explorer, selected_tiles, pre_footprint, post_footprint, final_pre, final_post)

m

In [ ]:
# @title Visualize selected pre- and post-disaster tile images

# Get unique combinations of date and catalog_id for post-disaster imagery
post_list = pd.unique(final_post[['date_post', 'catalog_id_post']].agg('_'.join, axis=1))

# Create a dropdown widget for selecting post-disaster imagery
post_picker = widgets.Dropdown(
    description='Select post-disaster imagery',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px'),
    options=post_list,
    value=post_list[0]
)

# Get unique combinations of date and catalog_id for pre-disaster imagery based on the selected post-disaster imagery
selected_post_catalog_id = post_list[0].split('_')[1]
pre_list = pd.unique(selected_tiles.loc[selected_tiles.catalog_id_post == selected_post_catalog_id, ['date_pre', 'catalog_id_pre']].agg('_'.join, axis=1))

# Create a dropdown widget for selecting pre-disaster imagery
pre_picker = widgets.Dropdown(
    description='Select pre-disaster imagery',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px'),
    options=pre_list,
    value=pre_list[0]
)

# Output widget for visualization
out_vis = widgets.Output()


class ImageComparisonWidget:
    def __init__(self, final_images, selected_post_id, selected_pre_id, all_post_images, all_pre_images,
                 intersected_aoi, main_widget):
        self.final_images = final_images
        self.selected_post_id = selected_post_id
        self.selected_pre_id = selected_pre_id
        self.main_widget = main_widget
        self.all_post_images = all_post_images
        self.all_pre_images = all_pre_images
        self.intersected_aoi = intersected_aoi

    def initialize_dropdown(self):
        out_vis.clear_output()
        self.setup_footprint()
        self.setup_tiles()
        self.setup_map()
        with out_vis:
            display(self.map)

    def setup_footprint(self):
        post_footprint = gpd.GeoDataFrame({'id': [1],
                                           'geometry': [self.all_post_images.loc[
                                                            self.all_post_images.catalog_id_post == self.selected_post_id.split('_')[1]].unary_union]})
        pre_footprint = gpd.GeoDataFrame({'id': [1],
                                          'geometry': [self.all_pre_images.loc[
                                                           self.all_pre_images.catalog_id_pre == self.selected_pre_id.split('_')[1]].unary_union]})
        if not (self.main_widget.area_of_interest == '' or self.main_widget.area_of_interest is None):
            self.intersected_aoi_post_pre = gpd.GeoDataFrame(
                post_footprint.intersection(pre_footprint).intersection(self.main_widget.gdf_aoi)).set_geometry(0)
        else:
            self.intersected_aoi_post_pre = gpd.GeoDataFrame(post_footprint.intersection(pre_footprint)).set_geometry(0)

    def setup_tiles(self):
        self.pre_stac_url = leafmap.maxar_tile_url(self.main_widget.mxr_coll, self.selected_pre_id.split('_')[1],
                                                   dtype='json')
        self.post_stac_url = leafmap.maxar_tile_url(self.main_widget.mxr_coll, self.selected_post_id.split('_')[1],
                                                    dtype='json')

    def setup_map(self):
        map_instance = leafmap.Map()

        if not (self.main_widget.area_of_interest == '' or self.main_widget.area_of_interest is None):
            map_instance.add_gdf(self.main_widget.gdf_aoi, layer_name="Area Of Interest",
                                 style={'color': 'orange', 'weight': 2, "fill": False})
        map_instance.add_gdf(self.intersected_aoi, layer_name="AOI Imagery Overlap",
                             style={'color': 'blue', 'weight': 2, "fill": False})
        map_instance.add_gdf(self.intersected_aoi_post_pre, layer_name="AOI Imagery Overlap Collection",
                             style={'color': 'cyan', 'weight': 2, "fill": False})

        map_instance.split_map(
            left_layer=self.pre_stac_url,
            right_layer=self.post_stac_url,
            left_label=f'Pre-event_{self.selected_pre_id}',
            right_label=f'Post-event_{self.selected_post_id}'
        )

        self.map = map_instance

    def select_dropdown_post(self, change):
        out_vis.clear_output()

        self.selected_post_id = change.new

        pre_list = pd.unique(self.final_images.loc[
            self.final_images.catalog_id_post == self.selected_post_id.split('_')[1],
            ['date_pre', 'catalog_id_pre']].agg('_'.join, axis=1))
        self.selected_pre_id = pre_list[0]

        if pre_picker.options != pre_list:
          pre_picker.options = pre_list
        else:
          self.setup_footprint()
          self.setup_tiles()
          self.setup_map()
          with out_vis:
            display(self.map)


    def select_dropdown_pre(self, change):
        out_vis.clear_output()

        self.selected_pre_id = change.new

        self.setup_footprint()
        self.setup_tiles()
        self.setup_map()

        with out_vis:
            display(self.map)

comparison_widget = ImageComparisonWidget(selected_tiles, post_picker.value, pre_picker.value,
                                          final_post, final_pre, intersection_pre_post_aoi, map_explorer)
comparison_widget.initialize_dropdown()

post_picker.observe(comparison_widget.select_dropdown_post, names="value")
pre_picker.observe(comparison_widget.select_dropdown_pre, names="value")

display(post_picker)
display(pre_picker)
display(out_vis)


In [ ]:
# @title Launch the download of the selected imagery from Maxar bucket

##@markdown ---
PATH_FILE_FOLDER = "" #@param {type:"string"}
#The destination path of the metadata csv file (colab path or gsutil URI):
PATH_FILE_LIST = os.path.join(PATH_FILE_FOLDER, 'metadata_maxarimagery.csv')
#The destination path of the AOI geojson file (colab path or gsutil URI):
PATH_FILE_AOI = os.path.join(PATH_FILE_FOLDER, 'aoi_maxarimagery.geojson')
#The destination paths of the folders to store the imagery tif files (colab path or gsutil URI):
PATH_FOLDER_PRE = os.path.join(PATH_FILE_FOLDER, 'pre_maxarimagery')
PATH_FOLDER_POST = os.path.join(PATH_FILE_FOLDER, 'post_maxarimagery')
PATH_FILE_CONFIG = os.path.join(PATH_FILE_FOLDER, 'config_file.json')

dict_config={"dataset_name": collection_picker.value}

if PATH_FILE_LIST!="":
  if PATH_FILE_LIST[:5]!='gs://':
    with open(PATH_FILE_LIST, 'w') as file:
        selected_tiles.to_csv(file)
  else:
    with open('/content/sample_data/imagery_list_maxar.csv', 'w') as file:
        selected_tiles.to_csv(file)
    auth.authenticate_user()
    os.system(f"""gsutil cp /content/sample_data/imagery_list_maxar.csv {PATH_FILE_LIST}""")

if PATH_FILE_AOI!="":
  dict_config["aoi_path"]=PATH_FILE_AOI
  if PATH_FILE_AOI[:5]!='gs://':
    intersection_pre_post_aoi.to_file(PATH_FILE_AOI, driver="GeoJSON")
  else:
    intersection_pre_post_aoi.to_file('/content/sample_data/aoi_intersect_maxar.geojson', driver="GeoJSON")
    auth.authenticate_user()
    os.system(f"""gsutil cp /content/sample_data/aoi_intersect_maxar.geojson {PATH_FILE_AOI}""")

print('Download of the pre-disaster imagery')
dict_config["before_image_patterns"]=[PATH_FOLDER_PRE]
download_images(final_pre.visual_pre, PATH_FOLDER_PRE)

print('Download of the post-disaster imagery')
dict_config["after_image_patterns"]=[PATH_FOLDER_POST]
download_images(final_post.visual_post, PATH_FOLDER_POST)

dict_config["resolution"]=0.3
dict_config["output_shards"]= 20

if PATH_FILE_CONFIG[:5]!='gs://':
    with open(PATH_FILE_CONFIG, 'w') as file:
        json.dump(dict_config, file)
else:
    with open('/content/sample_data/config_file.json', 'w') as file:
      json.dump(dict_config, file)
    os.system(f"""gsutil cp /content/sample_data/config_file.json {PATH_FILE_CONFIG}""")
